# POP MOC(z) for 0.1-degree
 - Computes MOC(z) offline from POP history files using simple xhistogram binning

In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import dask
from xhistogram.xarray import histogram
import pop_tools
import os
import time
import matplotlib.pyplot as plt
%matplotlib inline

from MOCutils import popmoc

In [2]:
import dask
from dask.distributed import wait
dask.__version__

'2022.11.0'

### Start Dask Cluster

In [14]:
# Close out Dask Cluster and release workers:
client.shutdown()

In [3]:
def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=1,
        memory='20GB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=1:mem=20GB',
        account='NCGD0011',
        walltime='06:00:00',
        local_directory='/glade/scratch/yeager/',
        log_directory='/glade/scratch/yeager/',
        interface='ib0',
        job_script_prologue=[
        f'export ESMFMKFILE={os.environ["ESMFMKFILE"]}'])

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status',
        "distributed.scheduler.worker-saturation": 1.0,
        'array.slicing.split_large_chunks': True
    })
    client = Client(cluster)
    return cluster, client

In [4]:
cluster, client = get_ClusterClient()
cluster.scale(60) 

/glade/work/yeager/conda-envs/mypythonenv/lib/python3.10/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36839 instead
  warnings.warn(


In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/yeager/proxy/36839/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/yeager/proxy/36839/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.34:33738,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/yeager/proxy/36839/status,Total threads: 0
Started: Just now,Total memory: 0 B


### 1. Read in Needed Grid Info

In [6]:
def time_set_midmonth(ds, time_name, deep=False):
    """
    Return copy of ds with values of ds[time_name] replaced with mid-month
    values (day=15) rather than end-month values.
    """
    #ds_out = ds.copy(deep)
    year = ds[time_name].dt.year
    month = ds[time_name].dt.month
    year = xr.where(month==1,year-1,year)
    month = xr.where(month==1,12,month-1)
    nmonths = len(month)
    newtime = [cftime.DatetimeNoLeap(year[i], month[i], 15) for i in range(nmonths)]
    ds[time_name] = newtime
    return ds

def preprocessor(ds0):
    keepvars = ['WVEL','VVEL']
    d0 = ds0[keepvars]
    d0 = time_set_midmonth(d0,'time')
    #d0 = d0.groupby('time.year').mean('time').rename({'year':'time'})
    return d0

# shift VVEL to 3121 position
def shiftVVEL(v):
    v_w = v.roll(nlon=1,roll_coords=False)
    v = 0.5*(v+v_w)
    return v.drop(['ULONG','TLAT'])

In [7]:
fgrd = '/glade/work/fredc/cesm/grid/POP/grid.3600x2400x62.nc'
ds_grid = xr.open_dataset(fgrd)

fmoc = '/glade/u/home/yeager/analysis/python/POP_MOC/moc_template.nc'
ds_moctemp = xr.open_dataset(fmoc)

In [8]:
%%time
dz = ds_grid['dz'].persist() / 100.
kmt = ds_grid['KMT'].fillna(0).persist() 
#dzt,dzu = popmoc.tx0p1v3_dztdzu(dz,kmt)

CPU times: user 111 ms, sys: 129 ms, total: 240 ms
Wall time: 373 ms


In [9]:
tlon = ds_grid.TLONG.drop(['ULONG','ULAT'])
tlat = ds_grid.TLAT.drop(['ULONG','ULAT'])
ulon = ds_grid.ULONG.drop(['TLONG','TLAT'])
ulat = ds_grid.ULAT.drop(['TLONG','TLAT'])

### MOC Region Mask

In [10]:
## Define the MOC region mask:
rmask = ds_grid.REGION_MASK.drop(['ULONG','ULAT'])
rmaskglob = xr.where((rmask>0),1,0)
rmaskatl = xr.where((rmask>=6) & (rmask<=11),1,0)
rmaskmoc = xr.concat([rmaskglob,rmaskatl],dim=ds_moctemp.transport_regions)

In [11]:
# determine j=index of Atlantic region southern boundary
tmp = rmaskmoc.isel(transport_reg=1).sum('nlon')
atl_j = 0
j = 0
while (atl_j==0):
    if (tmp.isel(nlat=j).data>0):
        atl_j = j
    j += 1
atl_j = atl_j - 1

# Loop over years and compute MOC 

In [12]:
ystart=89
yend=130
fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/HighResMIP/B1950cntl/HR/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013/ocn/proc/tseries/month_1/'
fin_tmp = 'B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.VVV.YYYY01-YYYY12.nc'
fin_tmp_out = 'B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.MOC.YYYY.nc'
outdir = '/glade/scratch/yeager/BSPNA'

In [13]:
%%time
for n in np.arange(ystart,yend+1,1):
    yrstr = str(n).zfill(4)
    fin = fdir+fin_tmp.replace('YYYY',yrstr).replace('VVV','VVEL')
    print("working on {}".format(fin))
    dsV = time_set_midmonth(xr.open_dataset(fin, chunks={'time':1,'nlon':100}),'time')
    fin = fdir+fin_tmp.replace('YYYY',yrstr).replace('VVV','WVEL')
    dsW = time_set_midmonth(xr.open_dataset(fin, chunks={'time':1,'nlon':100}),'time')
    
    w_e = dsW['WVEL'].drop(['ULONG','ULAT'])
    w_e = w_e.where(w_e<1.e30,0)
    v_e = shiftVVEL(dsV['VVEL'])
    v_e = v_e.where(v_e<1.e30,0)

    # Grid-oriented Volume FLuxes:
    tarea = ds_grid['TAREA'].drop(['ULONG','ULAT']).astype(w_e.dtype)
    w_e = (w_e*tarea/1.e6).assign_attrs({'units':'m^3/s'})
    dxdz = (ds_grid['HTN'].drop(['ULONG','TLAT'])*ds_grid['dz']).astype(v_e.dtype)
    v_e = (v_e*dxdz/1.e6).assign_attrs({'units':'m^3/s'})
    
    # add extra level corresponding to zeroes at seafloor (requires rechunking)
    w_e_bot = xr.zeros_like(w_e.isel(z_w_top=-1))
    w_e_bot['z_w_top'] = ds_grid.z_w_bot[-1].data
    wflux = xr.concat([w_e,w_e_bot],dim='z_w_top').rename({'z_w_top':'moc_z'}).chunk({'moc_z':63})
    
    v_e_bot = xr.zeros_like(v_e.isel(z_t=-1))
    v_e_bot['z_t'] = ds_grid.z_w_bot[-1].data
    vflux = xr.concat([v_e,v_e_bot],dim='z_t').chunk({'z_t':63})
    
    wflux = wflux.assign_coords({'TLONG':tlon, 'TLAT':tlat})
    vflux = vflux.assign_coords({'TLONG':tlon, 'ULAT':tlat})

    # Compute MOC
    MOC = popmoc.compute_MOC(wflux,rmaskmoc,ds_moctemp.lat_aux_grid)
    MOC = MOC.transpose('time','transport_reg','moc_z','lat_aux_grid')
    MOC = dask.optimize(MOC)[0]

    # add vflux at southern boundary of Atlantic domain
    tmp = vflux*(rmaskmoc.shift(nlat=-1))
    tmp = tmp.isel(nlat=atl_j,transport_reg=1).sum('nlon').rename({'z_t':'moc_z'})
    mocatl_s = -tmp.sortby('moc_z',ascending=False).cumsum('moc_z').sortby('moc_z',ascending=True)/1.e6
    mocatl_s['moc_z'] = MOC['moc_z']
    MOC[{'transport_reg':1}] = MOC[{'transport_reg':1}] + mocatl_s

    # Save to netcdf
    MOCann = MOC.groupby('time.year').mean('time').rename({'year':'time'})
    dsout = MOCann.to_dataset()

    fout = fin_tmp_out.replace('YYYY',yrstr)
    fout = os.path.join(outdir,fout)
    dsout.to_netcdf(fout,unlimited_dims='time')
    print("wrote {} at {}".format(fout, time.ctime()))

working on /glade/campaign/collections/cmip/CMIP6/iHESP/HighResMIP/B1950cntl/HR/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013/ocn/proc/tseries/month_1/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.VVEL.008901-008912.nc
wrote /glade/scratch/yeager/BSPNA/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.MOC.0089.nc at Sun Oct  1 08:10:58 2023
working on /glade/campaign/collections/cmip/CMIP6/iHESP/HighResMIP/B1950cntl/HR/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013/ocn/proc/tseries/month_1/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.VVEL.009001-009012.nc
wrote /glade/scratch/yeager/BSPNA/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.MOC.0090.nc at Sun Oct  1 08:12:16 2023
working on /glade/campaign/collections/cmip/CMIP6/iHESP/HighResMIP/B1950cntl/HR/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013/ocn/proc/tseries/month_1/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.VVEL.009101-009112.nc
wrote /glade/scratch/yeager/BSPNA/B.E.13.B1950C5.